In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast

import warnings

warnings.filterwarnings('ignore')

In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'drapery_length'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Curtains"""


# regex_pattern = r"(?i)(\s[^t]\d+\-Inch x \d+\-Inch)|()"
regex_pattern = r'(\s?\d*\s?\-\s?Inch\s?x?\s?\d*\-?\s?)()'


value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [4]:
df = query_from_file(file_name='query/bbby.sql', params=params)
df

attribute            value    customers_name external_id  \
0      drapery_length  101 in - 105 in  bedbathandbeyond    47556232   
1      drapery_length  106 in - 110 in  bedbathandbeyond    40497761   
2      drapery_length  106 in - 110 in  bedbathandbeyond    40557496   
3      drapery_length  106 in - 110 in  bedbathandbeyond    40935508   
4      drapery_length  106 in - 110 in  bedbathandbeyond    42993858   
...               ...              ...               ...         ...   
15532  drapery_length      Under 20 in  bedbathandbeyond    69608582   
15533  drapery_length      Under 20 in  bedbathandbeyond    69609675   
15534  drapery_length      Under 20 in  bedbathandbeyond    69609677   
15535  drapery_length      Under 20 in  bedbathandbeyond    69614804   
15536  drapery_length      Under 20 in  bedbathandbeyond    69689378   

                                                                                          long_desc  \
0                                         Sharper Image® 102-Inch Insulating Snap-In Liner in White   
1            Arm and Hammer Curtain Fresh Odor Neutralizing 108-Inch Sheer Curtain Panel in Latte   
2                          Cambria® Terra 108-Inch Rod Pocket Sheer Window Curtain Panel in Natural   
3                                        Alton Solid Grommet 108-Inch Window Curtain Panel in White   
4                 Kenneth Cole Gotham 108-Inch Grommet Room Darkening Window Curtain Panel in White   
...                                                                                             ...   
15532  Sun Zero Evelina Faux Dupioni Total Blackout 17-Inch Back Tab Window Curtain Valance in Gold   
15533                                         Clean Window® Twill Stripe Anti-Dust Valance in Mocha   
15534                                         Clean Window® Twill Stripe Anti-Dust Valance in White   
15535                                  J. Queen New York™ Delilah Straight Window Valance in Indigo   
15536                                              J. Queen New York™ Nouveau Window Valance in Spa   

      resolution  
0        invalid  
1        invalid  
2       standard  
3       standard  
4       standard  
...          ...  
15532    invalid  
15533    invalid  
15534    invalid  
15535    invalid  
15536    invalid  

[15537 rows x 6 columns]

In [5]:
metric='in'

range_string= """
 101 in - 105 in
106 in - 110 in
111 in - 115 in
116 in - 120 in
121 in - 125 in
126 in - 130 in
131 in - 135 in
136 in - 140 in
20 in - 25 in
26 in - 30 in
31 in - 35 in
36 in - 40 in
41 in - 45 in
46 in - 50 in
51 in - 55 in
56 in - 60 in
61 in - 65 in
66 in - 70 in
71 in - 75 in
76 in - 80 in
81 in - 85 in
86 in - 90 in
91 in - 95 in
96 in - 100 in
Above 140 in
Under 20 in 
"""

In [6]:
# # Find a list of values to iterate
# df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
# df_strategy
# list_values=df_strategy['value'].to_list()
# list_values

In [7]:
# kwp=KeywordProcessor()
# print(kwp.add_keywords_from_list(list_values))
# # Extract Key Words
# df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

In [8]:
# df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
# df['matches'].explode().value_counts()

In [9]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df

attribute            value    customers_name external_id  \
0      drapery_length  101 in - 105 in  bedbathandbeyond    47556232   
1      drapery_length  106 in - 110 in  bedbathandbeyond    40497761   
2      drapery_length  106 in - 110 in  bedbathandbeyond    40557496   
3      drapery_length  106 in - 110 in  bedbathandbeyond    40935508   
4      drapery_length  106 in - 110 in  bedbathandbeyond    42993858   
...               ...              ...               ...         ...   
15532  drapery_length      Under 20 in  bedbathandbeyond    69608582   
15533  drapery_length      Under 20 in  bedbathandbeyond    69609675   
15534  drapery_length      Under 20 in  bedbathandbeyond    69609677   
15535  drapery_length      Under 20 in  bedbathandbeyond    69614804   
15536  drapery_length      Under 20 in  bedbathandbeyond    69689378   

                                                                                          long_desc  \
0                                         Sharper Image® 102-Inch Insulating Snap-In Liner in White   
1            Arm and Hammer Curtain Fresh Odor Neutralizing 108-Inch Sheer Curtain Panel in Latte   
2                          Cambria® Terra 108-Inch Rod Pocket Sheer Window Curtain Panel in Natural   
3                                        Alton Solid Grommet 108-Inch Window Curtain Panel in White   
4                 Kenneth Cole Gotham 108-Inch Grommet Room Darkening Window Curtain Panel in White   
...                                                                                             ...   
15532  Sun Zero Evelina Faux Dupioni Total Blackout 17-Inch Back Tab Window Curtain Valance in Gold   
15533                                         Clean Window® Twill Stripe Anti-Dust Valance in Mocha   
15534                                         Clean Window® Twill Stripe Anti-Dust Valance in White   
15535                                  J. Queen New York™ Delilah Straight Window Valance in Indigo   
15536                                              J. Queen New York™ Nouveau Window Valance in Spa   

      resolution       matches  
0        invalid  [ 102-Inch ]  
1        invalid  [ 108-Inch ]  
2       standard  [ 108-Inch ]  
3       standard  [ 108-Inch ]  
4       standard  [ 108-Inch ]  
...          ...           ...  
15532    invalid   [ 17-Inch ]  
15533    invalid            []  
15534    invalid            []  
15535    invalid            []  
15536    invalid            []  

[15537 rows x 7 columns]

In [10]:
df=df[(df['value'].astype(str)=='n/a')&(df['matches'].astype(str)!='[]')]
df

attribute value    customers_name external_id  \
8972   drapery_length   n/a  bedbathandbeyond    10849217   
8975   drapery_length   n/a  bedbathandbeyond    12319185   
8986   drapery_length   n/a  bedbathandbeyond    14318593   
8988   drapery_length   n/a  bedbathandbeyond    14501614   
8991   drapery_length   n/a  bedbathandbeyond    14832432   
...               ...   ...               ...         ...   
15436  drapery_length   n/a  bedbathandbeyond    69683270   
15437  drapery_length   n/a  bedbathandbeyond    69683271   
15438  drapery_length   n/a  bedbathandbeyond    69683272   
15439  drapery_length   n/a  bedbathandbeyond    69683273   
15440  drapery_length   n/a  bedbathandbeyond    69683274   

                                                                               long_desc  \
8972   Redi Shade 36-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
8975   Redi Shade 48-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
8986                                          Dot Swirl 72-Inch x 72-Inch Shower Curtain   
8988                               Ivy Silver 70-Inch x 72-Inch EVA Vinyl Shower Curtain   
8991                                              Alyssa72-Inch Sidelight Panel in Ivory   
...                                                                                  ...   
15436                          Moda 71-Inch x 71-Inch Quaker Waffle Shower Curtain White   
15437                   Moda Spencer 71-Inch x 71-Inch Slub Shower Curtain in Grey/Taupe   
15438                             Moda On the Road 71-Inch x 71-Inch Peva Shower Curtain   
15439                 Moda Well Groomed 71-Inch x 71-Inch Shower Curtain in Orange/Taupe   
15440             Dainty Home 70-Inch x 72-Inch Sunrise Puckered Shower Curtain in White   

      resolution           matches  
8972    standard  [ 36-Inch x 72-]  
8975    standard  [ 48-Inch x 72-]  
8986    standard  [ 72-Inch x 72-]  
8988    standard  [ 70-Inch x 72-]  
8991    standard        [72-Inch ]  
...          ...               ...  
15436   standard  [ 71-Inch x 71-]  
15437   standard  [ 71-Inch x 71-]  
15438   standard  [ 71-Inch x 71-]  
15439   standard  [ 71-Inch x 71-]  
15440   standard  [ 70-Inch x 72-]  

[5319 rows x 7 columns]

In [11]:
df['matches'].explode().value_counts()

5-Inch x 72-      474
5-Inch x 64-      454
5-Inch x 48-      439
 72-Inch x 72-    274
 84-Inch          196
                 ... 
 58-Inch x 36-      1
 25-Inch            1
 58-Inch            1
 42-Inch            1
 132-Inch           1
Name: matches, Length: 410, dtype: int64

In [12]:
error

In [13]:
def split_col(data):
    data['matches']=data['matches'].str.get(0)
    # data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())
    data[['length', 'width']]=data['matches'].str.split("x",expand=True,)

In [14]:
def drop_characters(data):
    data['length']=data['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['width']=data['width'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('"', ''))
    data['width']=data['width'].apply(lambda quote: quote.replace('"', ''))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('-',''))
    data['width']=data['width'].apply(lambda quote: quote.replace('-',''))
    data['width']=data['width'].apply(lambda x: 0 if x == '' else x)
    data['length']=data['length'].apply(lambda x: 0 if x == '' else x)

In [15]:
def add_quotes(data):
    data['length']=data['length'].apply(lambda x: str(x) + "'")
    data['width']=data['width'].apply(lambda x: str(x) + "'")


In [16]:
def min_max(data):
    data['length']=data['length'].astype(int)
    data['width']=data['width'].astype(int)
    data['Max'] = data[['length', 'width']].values.max(1)
    data['Min'] = data[['length', 'width']].values.min(1)

In [17]:
split_col(df) #might throw an error and have to drop nan
drop_characters(df)
min_max(df) #error base 10 when there is only a value for max and not min

In [18]:
df

attribute value    customers_name external_id  \
8972   drapery_length   n/a  bedbathandbeyond    10849217   
8975   drapery_length   n/a  bedbathandbeyond    12319185   
8986   drapery_length   n/a  bedbathandbeyond    14318593   
8988   drapery_length   n/a  bedbathandbeyond    14501614   
8991   drapery_length   n/a  bedbathandbeyond    14832432   
...               ...   ...               ...         ...   
15436  drapery_length   n/a  bedbathandbeyond    69683270   
15437  drapery_length   n/a  bedbathandbeyond    69683271   
15438  drapery_length   n/a  bedbathandbeyond    69683272   
15439  drapery_length   n/a  bedbathandbeyond    69683273   
15440  drapery_length   n/a  bedbathandbeyond    69683274   

                                                                               long_desc  \
8972   Redi Shade 36-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
8975   Redi Shade 48-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
8986                                          Dot Swirl 72-Inch x 72-Inch Shower Curtain   
8988                               Ivy Silver 70-Inch x 72-Inch EVA Vinyl Shower Curtain   
8991                                              Alyssa72-Inch Sidelight Panel in Ivory   
...                                                                                  ...   
15436                          Moda 71-Inch x 71-Inch Quaker Waffle Shower Curtain White   
15437                   Moda Spencer 71-Inch x 71-Inch Slub Shower Curtain in Grey/Taupe   
15438                             Moda On the Road 71-Inch x 71-Inch Peva Shower Curtain   
15439                 Moda Well Groomed 71-Inch x 71-Inch Shower Curtain in Orange/Taupe   
15440             Dainty Home 70-Inch x 72-Inch Sunrise Puckered Shower Curtain in White   

      resolution         matches  length  width  Max  Min  
8972    standard   36-inch x 72-      36     72   72   36  
8975    standard   48-inch x 72-      48     72   72   48  
8986    standard   72-inch x 72-      72     72   72   72  
8988    standard   70-inch x 72-      70     72   72   70  
8991    standard        72-inch       72      0   72    0  
...          ...             ...     ...    ...  ...  ...  
15436   standard   71-inch x 71-      71     71   71   71  
15437   standard   71-inch x 71-      71     71   71   71  
15438   standard   71-inch x 71-      71     71   71   71  
15439   standard   71-inch x 71-      71     71   71   71  
15440   standard   70-inch x 72-      70     72   72   70  

[5319 rows x 11 columns]

In [20]:
def na_to_gsheet(df):
    url = 'https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0'
    wks = 'n/a'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    gc = gcloud.Gsheets()
    gc.save_to_gsheet(url, wks, non_matches)

In [21]:
na_to_gsheet(df)

succesfully saved to worksheet:`n/a` 
 url = `https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0`


In [22]:
# range function builder

string = range_string.replace(metric, '')
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)


def range_app(num):

    if num >=  101  and num <= 105 :
        return " 101  - 105 in"
        
    if num >= 106  and num <= 110 :
        return "106  - 110 in"
        
    if num >= 111  and num <= 115 :
        return "111  - 115 in"
        
    if num >= 116  and num <= 120 :
        return "116  - 120 in"
        
    if num >= 121  and num <= 125 :
        return "121  - 125 in"
        
    if num >= 126  and num <= 130 :
        return "126  - 130 in"
        
    if num >= 131  and num <= 135 :
        return "131  - 135 in"
        
    if num >= 136  and num <= 140 :
        return "136  - 140 in"
        
    if num >= 20  and num <= 25 :
        return "20  - 25 in"
        
    if num >= 26  and num <= 30 :
        return "26  - 30 in"
        
    if num >= 31  and num <= 35 :
        return "31  - 35 in"
        
    if num >= 36  and num <= 40 :
        return "36  - 40 in"
        
    if num >= 41  and num <= 45 :
        return "41  - 45 in"
        
    if n

In [23]:
matches=df
matches

attribute value    customers_name external_id  \
8972   drapery_length   n/a  bedbathandbeyond    10849217   
8975   drapery_length   n/a  bedbathandbeyond    12319185   
8986   drapery_length   n/a  bedbathandbeyond    14318593   
8988   drapery_length   n/a  bedbathandbeyond    14501614   
8991   drapery_length   n/a  bedbathandbeyond    14832432   
...               ...   ...               ...         ...   
15436  drapery_length   n/a  bedbathandbeyond    69683270   
15437  drapery_length   n/a  bedbathandbeyond    69683271   
15438  drapery_length   n/a  bedbathandbeyond    69683272   
15439  drapery_length   n/a  bedbathandbeyond    69683273   
15440  drapery_length   n/a  bedbathandbeyond    69683274   

                                                                               long_desc  \
8972   Redi Shade 36-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
8975   Redi Shade 48-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
8986                                          Dot Swirl 72-Inch x 72-Inch Shower Curtain   
8988                               Ivy Silver 70-Inch x 72-Inch EVA Vinyl Shower Curtain   
8991                                              Alyssa72-Inch Sidelight Panel in Ivory   
...                                                                                  ...   
15436                          Moda 71-Inch x 71-Inch Quaker Waffle Shower Curtain White   
15437                   Moda Spencer 71-Inch x 71-Inch Slub Shower Curtain in Grey/Taupe   
15438                             Moda On the Road 71-Inch x 71-Inch Peva Shower Curtain   
15439                 Moda Well Groomed 71-Inch x 71-Inch Shower Curtain in Orange/Taupe   
15440             Dainty Home 70-Inch x 72-Inch Sunrise Puckered Shower Curtain in White   

      resolution         matches  length  width  Max  Min  
8972    standard   36-inch x 72-      36     72   72   36  
8975    standard   48-inch x 72-      48     72   72   48  
8986    standard   72-inch x 72-      72     72   72   72  
8988    standard   70-inch x 72-      70     72   72   70  
8991    standard        72-inch       72      0   72    0  
...          ...             ...     ...    ...  ...  ...  
15436   standard   71-inch x 71-      71     71   71   71  
15437   standard   71-inch x 71-      71     71   71   71  
15438   standard   71-inch x 71-      71     71   71   71  
15439   standard   71-inch x 71-      71     71   71   71  
15440   standard   70-inch x 72-      70     72   72   70  

[5319 rows x 11 columns]

In [24]:
matches[curation_col] = matches['Max'].apply(range_app)
matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

attribute value    customers_name  external_id  \
8972   drapery_length   n/a  bedbathandbeyond  ="10849217"   
8975   drapery_length   n/a  bedbathandbeyond  ="12319185"   
8986   drapery_length   n/a  bedbathandbeyond  ="14318593"   
8988   drapery_length   n/a  bedbathandbeyond  ="14501614"   
8991   drapery_length   n/a  bedbathandbeyond  ="14832432"   
...               ...   ...               ...          ...   
15436  drapery_length   n/a  bedbathandbeyond  ="69683270"   
15437  drapery_length   n/a  bedbathandbeyond  ="69683271"   
15438  drapery_length   n/a  bedbathandbeyond  ="69683272"   
15439  drapery_length   n/a  bedbathandbeyond  ="69683273"   
15440  drapery_length   n/a  bedbathandbeyond  ="69683274"   

                                                                               long_desc  \
8972   Redi Shade 36-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
8975   Redi Shade 48-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
8986                                          Dot Swirl 72-Inch x 72-Inch Shower Curtain   
8988                               Ivy Silver 70-Inch x 72-Inch EVA Vinyl Shower Curtain   
8991                                              Alyssa72-Inch Sidelight Panel in Ivory   
...                                                                                  ...   
15436                          Moda 71-Inch x 71-Inch Quaker Waffle Shower Curtain White   
15437                   Moda Spencer 71-Inch x 71-Inch Slub Shower Curtain in Grey/Taupe   
15438                             Moda On the Road 71-Inch x 71-Inch Peva Shower Curtain   
15439                 Moda Well Groomed 71-Inch x 71-Inch Shower Curtain in Orange/Taupe   
15440             Dainty Home 70-Inch x 72-Inch Sunrise Puckered Shower Curtain in White   

      resolution         matches  length  width  Max  Min Q:drapery_length  
8972    standard   36-inch x 72-      36     72   72   36      71  - 75 in  
8975    standard   48-inch x 72-      48     72   72   48      71  - 75 in  
8986    standard   72-inch x 72-      72     72   72   72      71  - 75 in  
8988    standard   70-inch x 72-      70     72   72   70      71  - 75 in  
8991    standard        72-inch       72      0   72    0      71  - 75 in  
...          ...             ...     ...    ...  ...  ...              ...  
15436   standard   71-inch x 71-      71     71   71   71      71  - 75 in  
15437   standard   71-inch x 71-      71     71   71   71      71  - 75 in  
15438   standard   71-inch x 71-      71     71   71   71      71  - 75 in  
15439   standard   71-inch x 71-      71     71   71   71      71  - 75 in  
15440   standard   70-inch x 72-      70     72   72   70      71  - 75 in  

[5319 rows x 12 columns]

In [25]:
## KWP
# matches[curation_col]=matches['matches']
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [26]:
error----Switch Max and Min when doing length and width

SyntaxError: invalid syntax (<ipython-input-26-7509e0f4fe46>, line 1)

In [27]:
gc = gcloud.Gsheets()

In [28]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

succesfully saved to worksheet:`ul` 
 url = `https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0`


# send to the folder for upload

In [29]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [30]:
looks_good('Bed Bath & Beyond', attribute, df, matches)